In [ ]:
import pandas as pd
import time
import requests

### Read in Data & Calculate Height and Weight Percentiles

In [9]:
pitchers = pd.read_csv("pitchers.csv")

def height_to_inches(height_str):
    feet, inches = height_str.split("'")
    inches = inches.replace('"', '').strip()
    return int(feet) * 12 + int(inches)

pitchers['height_inches'] = pitchers['Ht'].apply(height_to_inches)

pitchers['height_percentile'] = pitchers['Ht'].rank(pct=True).round(2)
pitchers['weight_percentile'] = pitchers['Wt'].rank(pct=True).round(2)

pitchers_sorted = pitchers.sort_values(by='height_percentile', ascending=False)
print(pitchers_sorted)

# pitchers_sorted = pitchers.sort_values(by='weight_percentile', ascending=False)
# print(pitchers_sorted)


     Unnamed: 0               Name      Ht   Wt  height_inches  \
419         419        Bailey Ober   6' 9"  260             81   
130         130        Luke Little   6' 8"  220             80   
70           70     Félix Bautista   6' 8"  285             80   
641         641     Chris Roycroft   6' 8"  230             80   
77           77        Tyler Wells   6' 8"  260             80   
..          ...                ...     ...  ...            ...   
112         112      Shota Imanaga  5' 10"  175             70   
681         681     Danny Coulombe  5' 10"  190             70   
124         124         Eli Morgan  5' 10"  190             70   
716         716     Lazaro Estrada  5' 10"  180             70   
35           35  Daysbel Hernández  5' 10"  220             70   

     height_percentile  weight_percentile  
419               1.00               0.98  
130               0.99               0.65  
70                0.99               1.00  
641               0.99         